In [1]:
!pip install plotly

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objects as go # data viz
from plotly.subplots import make_subplots # create subplots
from ipywidgets import widgets # interactive data
from scipy.interpolate import UnivariateSpline # fitting
from statsmodels.tsa.arima_model import ARIMA # modeling
from sklearn.svm import SVR # modeling
from sklearn.linear_model import LinearRegression # linear modeling
from statsmodels.tsa.holtwinters import SimpleExpSmoothing # modeling
import time, datetime # work with date format
from datetime import timedelta # work with date format
import requests # get from API : https://developer.smartable.ai/api-details#api=coronavirus&operation=stats
import operator # sort list of sublist

In [3]:
raw_data = pd.read_csv("coronavirus.csv", header=0)
raw_data.replace({"Code": "US-DC"}, "US", inplace=True)
raw_data.head()

,Type,CountryOrRegion,ProvinceOrState,County,Code,Lat,Long,Source,1/22/2020,1/23/2020,...,3/9/2020,3/10/2020,3/11/2020,3/12/2020,3/13/2020,3/14/2020,3/15/2020,3/16/2020,3/17/2020,3/18/2020
0,Confirmed,Afghanistan,NaN,NaN,AF,33.0000,65.0000,NaN,0.0,0.0,...,4.0,5.0,7.0,7.0,7.0,16.0,16.0,21.0,22.0,22.0
1,Deaths,Afghanistan,NaN,NaN,AF,33.0000,65.0000,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Recovered,Afghanistan,NaN,NaN,AF,33.0000,65.0000,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
3,Confirmed,Albania,NaN,NaN,AL,41.1533,20.1683,NaN,0.0,0.0,...,2.0,10.0,12.0,23.0,33.0,38.0,42.0,51.0,55.0,59.0
4,Deaths,Albania,NaN,NaN,AL,41.1533,20.1683,NaN,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0


In [4]:
transformed_data_1 = raw_data[raw_data.columns[8: ]].fillna(0)
transformed_data_1 = transformed_data_1.astype(int)
transformed_data_1["CountryOrRegion"] = raw_data["CountryOrRegion"]
transformed_data_1["Type"] = raw_data["Type"]
transformed_data_2 = transformed_data_1.groupby(["CountryOrRegion", "Type"]).sum()
transformed_data_2.reset_index(inplace=True) 
transformed_data_2.rename(
    columns={
        date_string: datetime.datetime.strptime(date_string, "%m/%d/%Y") for date_string in transformed_data_2.columns[2:]
    }, inplace=True
)
transformed_data_2.head()

,CountryOrRegion,Type,2020-01-22 00:00:00,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,2020-01-29 00:00:00,...,2020-03-09 00:00:00,2020-03-10 00:00:00,2020-03-11 00:00:00,2020-03-12 00:00:00,2020-03-13 00:00:00,2020-03-14 00:00:00,2020-03-15 00:00:00,2020-03-16 00:00:00,2020-03-17 00:00:00,2020-03-18 00:00:00
0,Afghanistan,Confirmed,0,0,0,0,0,0,0,0,...,4,5,7,7,7,16,16,21,22,22
1,Afghanistan,Deaths,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Afghanistan,Recovered,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
3,Albania,Confirmed,0,0,0,0,0,0,0,0,...,2,10,12,23,33,38,42,51,55,59
4,Albania,Deaths,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,1,1,1,2


In [59]:
# Read from API
#api_url = "https://api.smartable.ai/coronavirus/stats/global"
#api_params = {
#    "Cache-Control": "no-cache",
#    "Subscription-Key": "f0850f2762ea4f658e6bb94d5ae23519",
#}
#r = requests.get(url=api_url, params=api_params) 
#data = r.json() 

#iso_codes = [element["location"]["isoCode"] for element in data["stats"]["breakdowns"] if element["location"]["isoCode"] != None ]
iso_codes = raw_data["Code"].drop_duplicates().dropna().values
api_data = pd.DataFrame()

for iso_code in iso_codes : 
    #print(f"Retrieving data from {iso_code}")
    api_url = f"https://api.smartable.ai/coronavirus/stats/{iso_code}"
    api_params = {
        "Cache-Control": "no-cache",
        "Subscription-Key": "f0850f2762ea4f658e6bb94d5ae23519",
    }
    r = requests.get(url=api_url, params=api_params) 
    data = r.json() 

    temp_data = pd.DataFrame(data["stats"]["history"])
    temp_data = temp_data.T
    temp_data.columns = temp_data.loc[temp_data.index == "date"].values[0]
    temp_data.drop(index="date", inplace=True)
    temp_data = temp_data.astype(int)
    temp_data.reset_index(inplace=True)
    temp_data.rename(
        columns={
            date_string: datetime.datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%S") for date_string in temp_data.columns[1:]
        }, inplace=True
    )
    temp_data.rename(columns={"index": "Type"}, inplace=True)
    temp_data["Type"] = temp_data["Type"].apply(lambda cell: cell.capitalize())
    temp_data["CountryOrRegion"] = raw_data["CountryOrRegion"].loc[raw_data["Code"] == iso_code].iloc[0]
    api_data = api_data.append(temp_data, ignore_index = True)


In [45]:
api_data.head()

,Type,2020-01-22 00:00:00,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,2020-01-29 00:00:00,2020-01-30 00:00:00,...,2020-03-31 00:00:00,2020-04-01 00:00:00,2020-04-02 00:00:00,2020-04-03 00:00:00,2020-04-04 00:00:00,2020-04-05 00:00:00,2020-04-06 00:00:00,2020-04-07 00:00:00,2020-04-08 00:00:00,CountryOrRegion
0,Confirmed,0,0,0,0,0,0,0,0,0,...,196,239,273,281,299,367,423,423,423,Afghanistan
1,Deaths,0,0,0,0,0,0,0,0,0,...,4,4,6,6,7,7,11,14,14,Afghanistan
2,Recovered,0,0,0,0,0,0,0,0,0,...,5,5,10,10,10,17,18,18,18,Afghanistan
3,Confirmed,0,0,0,0,0,0,0,0,0,...,243,259,277,304,333,361,377,383,383,Albania
4,Deaths,0,0,0,0,0,0,0,0,0,...,15,15,16,17,20,20,21,22,22,Albania


In [60]:
api_data = api_data.groupby(["CountryOrRegion", "Type"]).sum()
api_data.reset_index(inplace=True) 

In [62]:
last_date=api_data.columns[-2]
countries_whith_min_hundred_cases = api_data.loc[api_data[last_date] >= 100].copy()

# Note that the confirmed_cases count are the positive tested cases, and might be under estimated compared to real cases count
confirmed_cases = countries_whith_min_hundred_cases.loc[api_data["Type"] == "Confirmed"].copy()
confirmed_cases.set_index("CountryOrRegion", inplace=True)
confirmed_cases.drop(columns="Type", inplace=True)
confirmed_cases = confirmed_cases.sort_values(datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=1))

deceased = countries_whith_min_hundred_cases.loc[api_data["Type"] == "Deaths"].copy()
deceased.set_index("CountryOrRegion", inplace=True)
deceased.drop(columns="Type", inplace=True)

recovered = countries_whith_min_hundred_cases.loc[api_data["Type"] == "Recovered"].copy()
recovered.set_index("CountryOrRegion", inplace=True)
recovered.drop(columns="Type", inplace=True)

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=confirmed_cases.index,
        y=confirmed_cases[last_date],
        name="Confirmed",
        marker_color="rgb(55, 83, 109)"
    )
)
fig.add_trace(
    go.Bar(
        x=deceased.index,
        y=deceased[last_date],
        name="Deaths",
        marker_color="rgb(26, 118, 255)"
    )
)
fig.add_trace(
    go.Bar(
        x=recovered.index,
        y=recovered[last_date],
        name="Recovered",
        marker_color="rgb(255, 105, 180)"
    )
)
fig.update_layout(title=f"Count of confirmed cases the {last_date} (>= 100 cases)",
                  plot_bgcolor="rgb(255, 255, 255)")
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Confirmed cases count",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.show()

In [63]:
case_fatality = deceased[last_date] / (deceased[last_date] + recovered[last_date]) * 100
case_fatality.fillna(0, inplace=True)
case_fatality = case_fatality.loc[case_fatality.values != 0]
case_fatality.sort_values(inplace=True)
countries = case_fatality.index
case_recovery = [100 - fatality for fatality in case_fatality.values]

fig = go.Figure(data=[
    go.Bar(
        name='Recovered', 
        x=countries, 
        y=case_recovery,
        marker_color="rgb(0,128,0)",
        text=[f"{value:.1f}%" for value in case_recovery],
        textposition="inside"
    ),
    go.Bar(
        name='Passed', 
        x=countries, 
        y=case_fatality.values,
        marker_color="rgb(139,0,0)",
        text=[f"{value:.1f}%" for value in case_fatality.values],
        textposition="outside"
    )
])
fig.update_layout(title="Recovered/passed cases by country",
                  barmode="stack",
                  width=700, 
                  height=500, 
                  plot_bgcolor="rgb(255, 255, 255)")
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Ratio",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.show()

In [64]:
current_cases = confirmed_cases.subtract(recovered, fill_value=0)
current_cases = current_cases.sort_values(datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=1))

fig = go.Figure()
for country in countries_whith_min_hundred_cases["CountryOrRegion"].drop_duplicates().tolist():
    fig.add_trace(
        go.Scatter(
            x=current_cases.columns,
            y=current_cases[current_cases.index == country].values[0],
            name=country,
            mode="lines+markers",
            line={"width": 2}
        )
    )
fig.update_layout(title="Count of confirmed case (cases > 100)",
                  xaxis_tickformat="%d %B (%a)<br>%Y",
                  height=500, 
                  plot_bgcolor="rgb(255, 255, 255)")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Confirmed cases count",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.show()

In [65]:
day_count = 0
max_cases = 0

fig = go.Figure()
for country in current_cases.index:
    confirmed_cases_for_current_country = current_cases[current_cases.index == country].values[0]
    day_count_for_current_country = len(confirmed_cases_for_current_country)
    confirmed_cases_for_current_country = [date for date in confirmed_cases_for_current_country if date >= 100]
    max_cases_for_current_country = max(confirmed_cases_for_current_country)
    if day_count <= day_count_for_current_country:
        day_count = day_count_for_current_country
    if max_cases <= max_cases_for_current_country:
        max_cases = max_cases_for_current_country
    fig.add_trace(
        go.Scatter(
            x=np.arange(1, day_count_for_current_country + 1),
            y=confirmed_cases_for_current_country,
            name=country,
            mode="lines+markers",
            line={"width": 1}       
        )
    )
day_array = np.arange(1, day_count)
for r in [1, 2, 3, 5, 7]:
    if r == 1:
        text_value = "Double every day"
    else:
        text_value = f"Double every {r} days"
    trace=[100 * 2**(i/r) for i in day_array]
    fig.add_trace(
        go.Scatter(
            x=day_array,
            y=trace,
            name=text_value,
            mode="lines",
            line={"dash": "dash", "color": "MediumPurple", "width": 1},            
            showlegend=False
        )
    )
    text_x = np.argmax(np.array(trace) >= max_cases + 10000) if trace[-1] >= max_cases + 10000 else day_count - 4
    text_y = max_cases + 10000 if trace[-1] >= max_cases + 10000 else trace[-1]
    #fig.add_trace(
    #    go.Scatter(
    #        x=[text_x],
    #        y=[text_y],
    #        text=text_value,
    #        mode="text",
    #        showlegend=False
    #    )
    #)
fig.update_layout(
    title="Flattening the curve: when the 100th case is reached",
    xaxis={"range":(0, day_count + 4)},
    yaxis={"range":(-1000, max_cases + 15000)},
    height=500, 
    plot_bgcolor="rgb(255, 255, 255)")
fig.update_xaxes(title_text="Days since the 100th case",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.update_yaxes(title_text="Confirmed cases minus recovered",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.show()

In [66]:
france_confirmed_cases = confirmed_cases.loc[confirmed_cases.index == "France"]
china_confirmed_cases = confirmed_cases.loc[confirmed_cases.index == "China"]
italy_confirmed_cases = confirmed_cases.loc[confirmed_cases.index == "Italy"]
spain_confirmed_cases = confirmed_cases.loc[confirmed_cases.index == "Spain"]

france_deceased = deceased.loc[deceased.index == "France"]
china_deceased = deceased.loc[deceased.index == "China"]
italy_deceased = deceased.loc[deceased.index == "Italy"]
spain_deceased = deceased.loc[deceased.index == "Spain"]

france_recovered = recovered.loc[recovered.index == "France"]
china_recovered = recovered.loc[recovered.index == "China"]
italy_recovered = recovered.loc[recovered.index == "Italy"]
spain_recovered = recovered.loc[recovered.index == "Spain"]
france_confirmed_cases

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08
CountryOrRegion,,,,,,,,,,,,,,,,,,,,,
France,0,0,2,3,3,3,4,5,5,5,...,44550,52128,56989,59105,64338,89953,92839,98010,109069,109069


In [67]:
#New cases each day
fig = make_subplots(specs=[[{"secondary_y": True}]])
for cases_in_target_country in [current_cases.loc[current_cases.index=="China"]]:
    new_cases = cases_in_target_country.values[0].tolist()
    new_cases = [new_cases[i] - new_cases[i - 1] if i > 0 else 0 
                 for i in np.arange(0, len(new_cases))]
    fig.add_trace(
        go.Scatter(
            x=cases_in_target_country.columns,
            y=cases_in_target_country.values[0],
            name=cases_in_target_country.index[0],
            mode="lines+markers",
            marker={"color": "rgb(87, 46, 228)"}
        ),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(
            x=cases_in_target_country.columns,
            y=new_cases,
            name=cases_in_target_country.index[0],
            mode="lines",
            marker={"color": "rgb(231, 46, 1)"}
        ),
        secondary_y=True
    )
fig.update_layout(
    title="Confirmed cases evolution in China",
    showlegend=False,
    xaxis_tickformat="%d %B (%a)<br>%Y",
    height=500, 
    plot_bgcolor="rgb(255, 255, 255)")
fig.update_xaxes(title_text="Date",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.update_yaxes(title_text="Confirmed cases count", 
                 secondary_y=False,
                 color="rgb(87, 46, 228)",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.update_yaxes(title_text="New confirmed cases count", 
                 secondary_y=True,
                 color="rgb(231, 46, 1)")
fig.show()

In [35]:
new_confirmed_cases = confirmed_cases.loc[confirmed_cases[confirmed_cases.columns[-1]]>= 500].apply(lambda row: np.diff(row), axis=1)

fig = go.Figure()   
fig.add_trace(
    go.Heatmap(
        z=new_confirmed_cases.values,
        x=confirmed_cases.columns,
        y=new_confirmed_cases.index,
        colorscale=[
            [0, 'rgb(250, 250, 250)'],        #0
            [1./10000, 'rgb(200, 200, 200)'], #10
            [1./1000, 'rgb(150, 150, 150)'],  #100
            [1./100, 'rgb(100, 100, 100)'],   #1000
            [1./10, 'rgb(50, 50, 50)'],       #10000
            [1., 'rgb(0, 0, 0)'],             #100000
        ],
        colorbar={"tick0": 0, "tickmode": "array", "tickvals": [0, 1000, 10000, 100000]}
    )
)
fig.update_layout(
    title="New confirmed cases evolution",
    xaxis_tickformat="%d %B (%a)<br>%Y",
    height=800, 
    plot_bgcolor="rgb(255, 255, 255)")
fig.update_xaxes(title_text="Date",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.update_yaxes(title_text="New confirmed cases count",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.show()

In [68]:
first_confirmed_case_date = datetime.datetime.strptime("2019-11-17", "%Y-%m-%d")
china_lockdown_date = datetime.datetime.strptime("2020-01-23", "%Y-%m-%d")
china_peak_date = datetime.datetime.strptime("2020-02-23", "%Y-%m-%d")
days_between_lockdown_and_epidemic_peak = (china_peak_date-china_lockdown_date).days

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=current_cases.columns,
        y=china_confirmed_cases.values[0],
        name="Data",
        mode="markers",
    )
)

# Polynomial fitting
y = china_confirmed_cases.values
y = y.reshape(len(y[0]), )
x = np.arange(0, len(y))

max_cases = 0
x_new = np.linspace(x[0], x[-1], len(x))
for degree in np.arange(3, 5):
    coefs, residual, rank, singular_values, rcond = np.polyfit(x,y,degree, full = True)
    f = np.poly1d(coefs)
    y_new = f(x_new)
    max_cases = max(y_new) if max_cases <= max(y_new) else max_cases
    
    #poly = [f"{degree} poly: {v:.1f} * x^{i}" for i, v in enumerate(coefs[::-1])]
    #poly = " + ".join(poly)
    poly = f"{degree} poly fitting"
    
    fig.add_trace(
        go.Scatter(
            x=current_cases.columns,
            y=y_new,
            name=poly,
        )
    )


# SVR
x = x.reshape(-1, 1)
svr = SVR(kernel="rbf", C=10000, gamma=0.01, epsilon=.001)
y=svr.fit(x, y).predict(x)
fig.add_trace(
    go.Scatter(
        x=current_cases.columns,
        y=svr.fit(x, y).predict(x),
        name="SVR fitting"
    )
)
    
fig.add_shape(
    {
        "x0": first_confirmed_case_date,
        "x1": first_confirmed_case_date,
        "y0": 0,
        "y1": max_cases + 7000
    }
)
fig.add_shape(
    {
        "x0": china_lockdown_date,
        "x1": china_lockdown_date,
        "y0": 0,
        "y1": max_cases + 7000
    }
)
fig.add_shape(
    {
        "x0": china_peak_date,
        "x1": china_peak_date,
        "y0": 0,
        "y1": max_cases + 7000
    }
)
fig.add_trace(
    go.Scatter(
        x=[first_confirmed_case_date,
           china_lockdown_date, 
           china_peak_date],
        y=[max_cases + 10000,
           max_cases + 10000,
           max_cases + 10000],
        text=["First confirmed case",
            "China lockdown", 
            "Epidemic peak"],
        mode="text",    
        showlegend=False
    )
)

fig.update_layout(
    title="Fitting on China confirmed cases",
    xaxis={"range":(first_confirmed_case_date - timedelta(days=10), current_cases.columns[-1])},
    yaxis={"range":(-1500, max_cases + 15000)},
    showlegend=False,
    height=500, 
    plot_bgcolor="rgb(255, 255, 255)",    
    xaxis_tickformat="%d %B (%a)<br>%Y"
)
fig.update_xaxes(title_text="Date",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.update_yaxes(title_text="Confirmed cases count",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.show()

In [69]:
### Italy
italy_lockdown_date = datetime.datetime.strptime("2020-03-09 00:00:00", "%Y-%m-%d %H:%M:%S")
italy_epidemic_putative_peak = italy_lockdown_date + timedelta(days=days_between_lockdown_and_epidemic_peak)
forecast_period = (italy_epidemic_putative_peak - italy_confirmed_cases.columns[-2]).days

## Confirmed cases forecasting
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(
        x=italy_confirmed_cases.columns[:-1],
        y=italy_confirmed_cases.values[0][:-1],
        name="Data",
        mode="markers",
        marker={"color": "rgb(87, 46, 228)"}
    ),
    secondary_y=False
)

# Autoregressive Integrated Moving Average (ARIMA)
x = italy_confirmed_cases.columns[:-1]
y = italy_confirmed_cases[x].astype("float32")
x_new = [italy_confirmed_cases.columns[-2] + timedelta(days=i) for i in range(0, forecast_period + 1)]

model = ARIMA(y.T.asfreq("D"), order=(1, 2, 0))
model_fit = model.fit(disp=False)
arima_model = model_fit.predict(x[-1], x_new[-1], typ='levels')

fig.add_trace(
    go.Scatter(
        x=x_new,
        y=arima_model,
        name="ARIMA",
        mode="lines",
        marker={"color": "rgb(87, 46, 228)"}
    ),
    secondary_y=False
)

# Linear
y = italy_confirmed_cases.values[0][:-1]
a = y[-1] - y[-2]
b = y[-1]
y_new = [a*x + b for x in np.arange(len(x_new))]

fig.add_trace(
    go.Scatter(
        x=x_new,
        y=y_new,
        name="Linear",
        mode="lines",
        marker={"color": "rgb(87, 46, 228)"}
    ),
    secondary_y=False
)

max_cases = max([max(arima_model), max(y_new)])

## Deceased
fig.add_trace(
    go.Scatter(
        x=italy_deceased.columns[:-1],
        y=italy_deceased.values[0][:-1],
        name="Data",
        mode="markers",
        marker={"color": "rgb(231, 46, 1)"}
    ),
    secondary_y=True
)

# Autoregressive Integrated Moving Average (ARIMA)
x = italy_deceased.columns[:-1]
y = italy_deceased[x].astype("float32")

model = ARIMA(y.T.asfreq("D"), order=(1, 2, 0))
model_fit = model.fit(disp=False)
arima_model = model_fit.predict(x[-1], x_new[-1], typ='levels')

fig.add_trace(
    go.Scatter(
        x=x_new,
        y=arima_model,
        name="ARIMA",
        mode="lines",
        marker={"color": "rgb(231, 46, 1)"}
    ),
    secondary_y=True
)

# Linear
y = italy_deceased.values[0][:-1]
a = y[-1] - y[-2]
b = y[-1]
y_new = [a*x + b for x in np.arange(len(x_new))]

fig.add_trace(
    go.Scatter(
        x=x_new,
        y=y_new,
        name="Linear",
        mode="lines",
        marker={"color": "rgb(231, 46, 1)"}
    ),
    secondary_y=True
)

fig.add_shape(
    {
        "x0": italy_lockdown_date,
        "x1": italy_lockdown_date,
        "y0": 0,
        "y1": max_cases + 7000
    }
)
fig.add_shape(
    {
        "x0": italy_epidemic_putative_peak,
        "x1": italy_epidemic_putative_peak,
        "y0": 0,
        "y1": max_cases + 7000
    }
)
fig.add_trace(
    go.Scatter(
        x=[italy_lockdown_date, 
           italy_epidemic_putative_peak],
        y=[max_cases + 10000,
           max_cases + 10000,
           max_cases + 10000],
        text=["Italy lockdown", 
            "Epidemic peak"],
        mode="text",    
        showlegend=False
    )
)

fig.update_layout(
    title="Forecasting Italy confirmed cases and deaths",
    xaxis={"range":(italy_confirmed_cases.columns[0], italy_confirmed_cases.columns[-1] + timedelta(days=days_between_lockdown_and_epidemic_peak))},
    yaxis={"range":(-1500, max_cases + 15000)},
    yaxis2={"range":(-1500, max_cases + 15000)},
    showlegend=False,
    height=500, 
    plot_bgcolor="rgb(255, 255, 255)",    
    xaxis_tickformat="%d %B (%a)<br>%Y",
)
fig.update_xaxes(title_text="Day",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.update_yaxes(title_text="Confirmed cases count", 
                 secondary_y=False,
                 color="rgb(87, 46, 228)",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.update_yaxes(title_text="Deaths count", 
                 secondary_y=True,
                 color="rgb(231, 46, 1)",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.show()

In [70]:
### France
france_lockdown_date = datetime.datetime.strptime("2020-03-17 00:00:00", "%Y-%m-%d %H:%M:%S")
france_epidemic_putative_peak = france_lockdown_date + timedelta(days=days_between_lockdown_and_epidemic_peak)
forecast_period = (france_epidemic_putative_peak - france_confirmed_cases.columns[-2]).days
france_intensive_care_capacity = 4566 # source: https://pandemy.info/

## Confirmed cases forecasting
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(
        x=france_confirmed_cases.columns[:-1],
        y=france_confirmed_cases.values[0][:-1],
        name="Data",
        mode="markers",
        marker={"color": "rgb(87, 46, 228)"}
    ),
    secondary_y=False
)

# Autoregressive Integrated Moving Average (ARIMA)
x = france_confirmed_cases.columns[:-1]
y = france_confirmed_cases[x].astype("float32")
x_new = [france_confirmed_cases.columns[-2] + timedelta(days=i) for i in range(0, forecast_period + 1)]

model = ARIMA(y.T.asfreq("D"), order=(1, 2, 0))
model_fit = model.fit(disp=False)
arima_model = model_fit.predict(x[-1], x_new[-1], typ='levels')

fig.add_trace(
    go.Scatter(
        x=x_new,
        y=arima_model,
        name="ARIMA",
        mode="lines",
        marker={"color": "rgb(87, 46, 228)"}
    ),
    secondary_y=False
)

# Linear
y = france_confirmed_cases.values[0][:-1]
a = y[-1] - y[-2]
b = y[-1]
y_new = [a*x + b for x in np.arange(len(x_new))]

fig.add_trace(
    go.Scatter(
        x=x_new,
        y=y_new,
        name="Linear",
        mode="lines",
        marker={"color": "rgb(87, 46, 228)"}
    ),
    secondary_y=False
)

max_cases= max([max(arima_model), max(y_new)])

## Deceased
fig.add_trace(
    go.Scatter(
        x=france_deceased.columns[:-1],
        y=france_deceased.values[0][:-1],
        name="Data",
        mode="markers",
        marker={"color": "rgb(231, 46, 1)"}
    ),
    secondary_y=True
)

# Autoregressive Integrated Moving Average (ARIMA)
x = france_deceased.columns[:-1]
y = france_deceased[x].astype("float32")

model = ARIMA(y.T.asfreq("D"), order=(1, 2, 0))
model_fit = model.fit(disp=False)
arima_model = model_fit.predict(x[-1], x_new[-1], typ='levels')

fig.add_trace(
    go.Scatter(
        x=x_new,
        y=arima_model,
        name="ARIMA",
        mode="lines",
        marker={"color": "rgb(231, 46, 1)"}
    ),
    secondary_y=True
)

# Linear
y = france_deceased.values[0][:-1]
a = y[-1] - y[-2]
b = y[-1]
y_new = [a*x + b for x in np.arange(len(x_new))]

fig.add_trace(
    go.Scatter(
        x=x_new,
        y=y_new,
        name="Linear",
        mode="lines",
        marker={"color": "rgb(231, 46, 1)"}
    ),
    secondary_y=True
)

fig.add_shape(
    {
        "x0": france_lockdown_date,
        "x1": france_lockdown_date,
        "y0": 0,
        "y1": max_cases + 7000
    }
)
fig.add_shape(
    {
        "x0": france_epidemic_putative_peak,
        "x1": france_epidemic_putative_peak,
        "y0": 0,
        "y1": max_cases + 7000
    }
)
fig.add_trace(
    go.Scatter(
        x=[france_lockdown_date, 
           france_epidemic_putative_peak],
        y=[max_cases + 10000,
           max_cases + 10000,
           max_cases + 10000],
        text=["France lockdown", 
            "Epidemic peak"],
        mode="text",    
        showlegend=False
    )
)

fig.update_layout(
    title="Forecasting France confirmed cases and deaths",
    xaxis={"range":(italy_confirmed_cases.columns[0], italy_confirmed_cases.columns[-1] + timedelta(days=days_between_lockdown_and_epidemic_peak))},
    yaxis={"range":(-1500, max_cases + 15000)},
    yaxis2={"range":(-1500, max_cases + 15000)},
    showlegend=False,
    height=500, 
    plot_bgcolor="rgb(255, 255, 255)",    
    xaxis_tickformat="%d %B (%a)<br>%Y",
)
fig.update_xaxes(title_text="Day",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.update_yaxes(title_text="Confirmed cases count", 
                 secondary_y=False,
                 color="rgb(87, 46, 228)",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.update_yaxes(title_text="Deaths count", 
                 secondary_y=True,
                 color="rgb(231, 46, 1)",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.show()

In [80]:
new_deaths = deceased.loc[deceased[deceased.columns[-1]]>= 100].apply(lambda row: np.diff(row), axis=1)

fig = go.Figure()   
fig.add_trace(
    go.Heatmap(
        z=new_deaths.values,
        x=deceased.columns,
        y=new_deaths.index,
        colorscale=[
            [0, 'rgb(250, 250, 250)'],        #0
            [1./10000, 'rgb(200, 200, 200)'], #10
            [1./1000, 'rgb(150, 150, 150)'],  #100
            [1./100, 'rgb(100, 100, 100)'],   #1000
            [1./10, 'rgb(50, 50, 50)'],       #10000
            [1., 'rgb(0, 0, 0)'],             #100000
        ],
        colorbar={"tick0": 0, "tickmode": "array", "tickvals": [0, 1000, 10000, 100000]}
    )
)
fig.update_layout(
    title="New deaths evolution",
    xaxis_tickformat="%d %B (%a)<br>%Y",
    height=800, 
    plot_bgcolor="rgb(255, 255, 255)")
fig.update_xaxes(title_text="Date",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.update_yaxes(title_text="New deaths count",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.show()

In [71]:
# This variable can be changed to explore the forecast
putative_lockdown_date = datetime.datetime.strptime("2020-03-25 00:00:00", "%Y-%m-%d %H:%M:%S")

day_before_putative_lockdown_date = putative_lockdown_date - timedelta(days=1)
putatative_peak_date = putative_lockdown_date + timedelta(days=days_between_lockdown_and_epidemic_peak)

fig = make_subplots(specs=[[{"secondary_y": True}]])
confirmed_cases_forecasts = []
for country in confirmed_cases.loc[confirmed_cases[confirmed_cases.columns[-1]] >= 100].index:
    current_country_data = confirmed_cases.loc[confirmed_cases.index == country]
    #a = current_country_data[putative_lockdown_date].values[0] - current_country_data[day_before_putative_lockdown_date].values[0]
    #b = current_country_data[putative_lockdown_date].values[0]
    #y = a * forecast_period + b
    
    x = current_country_data.columns[:-1]
    y = current_country_data[x].astype("float32")
    model = ARIMA(y.T.asfreq("D"), order=(1, 2, 0))
    model_fit = model.fit(disp=False)
    forecast = model_fit.predict(putative_lockdown_date, putatative_peak_date, typ='levels')[-1]
    confirmed_cases_forecasts.append([country, forecast])

max_cases = max([element[1] for element in confirmed_cases_forecasts])
    
deceased_forecasts = []
for country in deceased.loc[deceased[current_cases.columns[-1]] >= 100].index:
    current_country_data = deceased.loc[deceased.index == country]
    #a = current_country_data[putative_lockdown_date].values - current_country_data[day_before_putative_lockdown_date].values[0]
    #b = current_country_data[putative_lockdown_date].values
    #forecast = a * forecast_period + b
    
    x = current_country_data.columns[:-1]
    y = current_country_data[x].astype("float32")
    model = ARIMA(y.T.asfreq("D"), order=(1, 2, 0))
    model_fit = model.fit(disp=False)
    forecast = model_fit.predict(putative_lockdown_date, putatative_peak_date, typ='levels')[-1]
    deceased_forecasts.append([country, forecast])
        
fig.add_trace(
    go.Bar(
        name="Confirmed cases",
        x=[element[0] for element in sorted(confirmed_cases_forecasts, key=operator.itemgetter(1)) if element[1] >= 0],
        y=[element[1] for element in sorted(confirmed_cases_forecasts, key=operator.itemgetter(1)) if element[1] >= 0],
        marker_color="rgb(87, 46, 228)"
    ),
    secondary_y=False
)
fig.add_trace(
    go.Bar(
        name="Deceased",
        x=[element[0] for element in deceased_forecasts if element[1] >= 0],
        y=[element[1] for element in deceased_forecasts if element[1] >= 0],
        marker_color="rgb(231, 46, 1)"
    ),
    secondary_y=True
)

fig.update_layout(
    title=f"Forecasting confirmed cases and deaths in all country (if lockdown date was {putative_lockdown_date})", 
    showlegend=False,
    height=500, 
    plot_bgcolor="rgb(255, 255, 255)",   
    yaxis={"range":(-1500, max_cases + 15000)},
    yaxis2={"range":(-1500, max_cases + 15000)},
)
fig.update_yaxes(title_text="Confirmed cases forecast", 
                 secondary_y=False,
                 color="rgb(87, 46, 228)",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.update_yaxes(title_text="Deaths forecast", 
                 secondary_y=True,
                 color="rgb(231, 46, 1)",
                 gridcolor="rgb(200, 200, 200)", 
                 gridwidth=1)
fig.show()

In [72]:
# Forecast of global deaths 
"{:,}".format(int(sum([element[1] for element in deceased_forecasts])))

'218,013'

In [57]:
# Age distibution 
# Source : http://data.un.org/Data.aspx?d=POP&f=tableCode%3A22
age_distribution_raw_data = pd.read_excel("age_distribution.xlsx", header=0)
age_distribution_raw_data.dropna(inplace=True)

age_distribution = pd.DataFrame(columns=["Country or Area", "Sex", "Age", "Value", "Year"])
for country in age_distribution_raw_data["Country or Area"].drop_duplicates():
    data_for_current_country = age_distribution_raw_data.loc[age_distribution_raw_data["Country or Area"] == country]
    female_data = data_for_current_country.loc[data_for_current_country["Sex"] == "Female"]
    male_data = data_for_current_country.loc[data_for_current_country["Sex"] == "Male"]
    for sex_data_set in [female_data, male_data]:
        sex = sex_data_set["Sex"].iloc[0]
        year = int(sex_data_set["Year"].unique()[0])
        age_distribution = age_distribution.append(
            pd.Series(
                {"Country or Area": country,
                 "Sex": sex,
                 "Age": "0 - 9",
                 "Value": int(sum(sex_data_set["Value"].loc[(sex_data_set["Age"] == "0 - 4") | (sex_data_set["Age"] == "5 - 9")])),
                 "Year": year}
            ),
            ignore_index=True
        )
        for i in np.arange(1, 8):
            age_distribution = age_distribution.append(
                pd.Series(
                    {"Country or Area": country,
                     "Sex": sex,
                     "Age": f"{i}0 - {i}9",
                     "Value": int(sum(sex_data_set["Value"].loc[(sex_data_set["Age"] == f"{i}0 - {i}4") | (sex_data_set["Age"] == f"{i}5 - {i}9")])),
                     "Year": year}
                ),
                ignore_index=True
            )  
        age_distribution = age_distribution.append(
            pd.Series(
                {"Country or Area": country,
                 "Sex": sex,
                 "Age": f"80 +",
                 "Value": int(sum(sex_data_set["Value"].loc[(sex_data_set["Age"] == f"80 - 84") | (sex_data_set["Age"] == f"85 +")])),
                 "Year": year}
            ),
            ignore_index=True
        )

# Fatality by age 
# Source : http://weekly.chinacdc.cn/en/article/id/e53946e2-c6c4-41e9-9a9b-fea8db1a8f51
fatality_per_age_category = pd.DataFrame(
    [{"Age": "0 - 9", "Fatality rate": 0}, 
     {"Age": "10 - 19", "Fatality rate": 0.2}, 
     {"Age": "20 - 29", "Fatality rate": 0.2}, 
     {"Age": "30 - 39", "Fatality rate": 0.2}, 
     {"Age": "40 - 49", "Fatality rate": 0.4}, 
     {"Age": "50 - 59", "Fatality rate": 1.3}, 
     {"Age": "60 - 69", "Fatality rate": 3.6}, 
     {"Age": "70 - 79", "Fatality rate": 8.0}, 
     {"Age": "80 +", "Fatality rate": 14.8}],
    columns=["Age", "Fatality rate"]
)

In [43]:
y = age_distribution["Age"].drop_duplicates()

def death_by_age_distribution(country="France"):
    female_x = age_distribution["Value"].loc[(age_distribution["Country or Area"] == country) & (age_distribution["Sex"] == "Female")]
    male_x = age_distribution["Value"].loc[(age_distribution["Country or Area"] == country) & (age_distribution["Sex"] == "Male")]
    max_pop = max([max(female_x), max(male_x)])
    female_deaths_x = [int(-1 * round(i*j/100, 1)) for i, j in zip(female_x, fatality_per_age_category["Fatality rate"])]
    male_deaths_x = [int(round(i*j/100, 1)) for i, j in zip(male_x, fatality_per_age_category["Fatality rate"])]
    year = age_distribution["Year"].loc[age_distribution["Country or Area"] == country].unique()[0]
    return female_x, male_x, max_pop, female_deaths_x, male_deaths_x, year

female_x, male_x, max_pop, female_deaths_x, male_deaths_x, year = death_by_age_distribution()

# Age distribution
trace1 = go.Bar(
        x=(-1 * female_x),
        y=y,
        name="Female",
        orientation="h",
        text=["{:,}".format(x) for x in female_x],
        hoverinfo="text")
trace2 = go.Bar(
        x=male_x,
        y=y,
        name="Male",
        orientation="h",        
        text=["{:,}".format(x) for x in male_x],
        hoverinfo="text")
trace3 = go.Bar(
        x=female_deaths_x,
        y=y,
        name="Female deaths",
        orientation="h",
        text=["{:,}".format(-1 * x) for x in female_deaths_x],
        hoverinfo="text")
trace4 = go.Bar(
        x=male_deaths_x,
        y=y,
        name="Male deaths",
        orientation="h",        
        text=["{:,}".format(x) for x in male_deaths_x],
        hoverinfo="text")

ticks = np.arange((-1 * max_pop), max_pop, 10000000)
total_pop = "{:,}".format(sum(female_x) + sum(male_x))
total_deaths = "{:,}".format(int(-1 * sum(female_deaths_x) + sum(male_deaths_x)))

layout = go.Layout(
    title={"text": f"Deaths by age distribution in France<br>Total population: {total_pop} ({year})<br>Total deaths (if no lockdown): {total_deaths}"},
    barmode="overlay",
    bargap=0.1,
    #height=500, 
    plot_bgcolor="rgb(255, 255, 255)",
    xaxis={
        "title": "Count",
        "range": [(-1 * max_pop - 1000000), (max_pop + 1000000)],
        "showticklabels": False}
)

# Dynamic data, chosing the country to show
g = go.FigureWidget(data = [trace1, trace2, trace3, trace4], 
                    layout=layout)
country_dropdown = widgets.Dropdown(
    options=list(age_distribution["Country or Area"].unique()),
    value="France",
    description="Country:",
)

def response(change):
    female_x, male_x, max_pop, female_deaths_x, male_deaths_x, year = death_by_age_distribution(country_dropdown.value)
    total_pop = "{:,}".format(sum(female_x) + sum(male_x))
    total_deaths = "{:,}".format(int(-1 * sum(female_deaths_x) + sum(male_deaths_x)))
    with g.batch_update():
        g.data[0].x = (-1 * female_x)
        g.data[0].text = ["{:,}".format(1 * x) for x in female_x]
        g.data[1].x = male_x
        g.data[1].text = ["{:,}".format(x) for x in male_x]
        g.data[2].x = female_deaths_x
        g.data[2].text = ["{:,}".format(-1 * x) for x in female_deaths_x]
        g.data[3].x = male_deaths_x
        g.data[3].text = ["{:,}".format(x) for x in male_deaths_x]
        g.layout.title = f"Deaths by age distribution in {country_dropdown.value}<br>Total population: {total_pop} ({year})<br>Total deaths (if no lockdown): {total_deaths}"
        g.layout.xaxis.range = [(-1 * max_pop - 1000), (max_pop + 1000)]
    return None
            
country_dropdown.observe(response, names="value")

container = widgets.HBox([country_dropdown])
widgets.VBox([container, g])

In [58]:
# Compartmental model (toy model)
# Source : https://sciencetonnante.wordpress.com/
# R0 = C * P * D
# C: contact count with other people each day
# P: probability to be sick after being in contact with someone who is infected
# D: is the duration of contagion in days

time_range = np.arange(1, 180, 1)

def toy_model(C=25, P=0.0025, D=10, fatality_rate=0.03, initial_real_cases=2000, initial_recovered=200, initial_deceased=50, country="France", time_range=np.arange(1, 180, 1)) -> (list, list, list, list):
    R_zero = C * P * D # count of total people infected
    population = sum(age_distribution["Value"].loc[age_distribution["Country or Area"] == country])
    initial_healthy_people = population - initial_real_cases - initial_recovered - initial_deceased

    healthy_people = [initial_healthy_people]
    real_cases = [initial_real_cases]
    recovered = [initial_recovered]
    deceased = [initial_deceased]

    for i in time_range: 
        real_cases.append(real_cases[i-1] + (real_cases[i-1] * C * P * (healthy_people[i-1]/population)) - (1/D * real_cases[i-1]) - (fatality_rate/D * real_cases[i-1]))
        recovered.append(recovered[i-1] + (1/D * real_cases[i]))
        deceased.append(deceased[i-1] + (fatality_rate/D * real_cases[i]))
        healthy_people.append(population - real_cases[i] - recovered[i] - deceased[i])
    return healthy_people, real_cases, recovered, deceased

healthy_people, real_cases, recovered, deceased = toy_model()

trace1 = go.Scatter(
    x=time_range,
    y=real_cases,
    name="Real cases",
    text=["{:,}".format(element) for element in real_cases],
    hoverinfo="text")

trace2 = go.Scatter(
    x=time_range,
    y=recovered,
    name="Recovered",
    text=["{:,}".format(element) for element in recovered],
    hoverinfo="text")

trace3 = go.Scatter(
    x=time_range,
    y=deceased,
    name="Deceased",
    text=["{:,}".format(element) for element in deceased],
    hoverinfo="text")

layout = go.Layout(
    title={"text": f"Compartment model in France"},
    plot_bgcolor="rgb(255, 255, 255)",
    xaxis={"title": "Day", "gridcolor": "rgb(200, 200, 200)", "gridwidth": 1},
    yaxis={"title": "Count", "gridcolor": "rgb(200, 200, 200)", "gridwidth": 1}
)

g2 = go.FigureWidget(data = [trace1, trace2, trace3],
                     layout = layout)
country_dropdown_2 = widgets.Dropdown(
    options=list(age_distribution["Country or Area"].unique()),
    value="France",
    description="Country:",
)
initial_real_cases_textbox = widgets.Text(
    description="Initial real cases",
    value="2000",
)
initial_recovered_textbox = widgets.Text(
    description="Initial recovered",
    value="200",
)
initial_deceased_textbox = widgets.Text(
    description="Initial deceased",
    value="50",
)
contact_slider = widgets.FloatSlider(
    value=30,
    min=1,
    max=50,
    step=1,
    description="Contact each day",
    continuous_update = False
)
probability_slider = widgets.FloatSlider(
    value=0.0025,
    min=0.0,
    max=0.1,
    step=0.0005,
    description="Probability of contagion",
    continuous_update = True
)
duration_slider = widgets.FloatSlider(
    value=10,
    min=5,
    max=21,
    step=1,
    description="Duration of contagion",
    continuous_update = True
)
fatality_rate_slider = widgets.FloatSlider(
    value=0.03,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Fatality rate",
    continuous_update = True
)

def response(change):
    healthy_people, real_cases, recovered, deceased = toy_model(
        contact_slider.value, 
        probability_slider.value, 
        duration_slider.value, 
        fatality_rate_slider.value, 
        int(initial_real_cases_textbox.value), 
        int(initial_recovered_textbox.value),
        int(initial_deceased_textbox.value),
        country_dropdown_2.value, 
        time_range)
    with g2.batch_update():
        g2.data[0].y = real_cases
        g2.data[0].text = ["{:,}".format(x) for x in real_cases]
        g2.data[1].y = recovered
        g2.data[1].text = ["{:,}".format(x) for x in recovered]
        g2.data[2].y = deceased
        g2.data[2].text = ["{:,}".format(x) for x in deceased]
        g2.layout.title = f"Compartment model in {country_dropdown_2.value}"
    return None
     
country_dropdown_2.observe(response, names="value")
initial_real_cases_textbox.observe(response, names="value")
initial_recovered_textbox.observe(response, names="value")
initial_deceased_textbox.observe(response, names="value")
contact_slider.observe(response, names="value")
probability_slider.observe(response, names="value")
duration_slider.observe(response, names="value")
fatality_rate_slider.observe(response, names="value")

container = widgets.HBox([country_dropdown_2])
container_2 = widgets.HBox([initial_real_cases_textbox, initial_recovered_textbox])
container_3 = widgets.HBox([initial_deceased_textbox])
container_4 = widgets.HBox([contact_slider, probability_slider])
container_5 = widgets.HBox([duration_slider, fatality_rate_slider])
widgets.VBox([container, container_2, container_3, container_4, container_5, g2])